# Use of Tensorboard 

This notebook provides a basic structure to use tensorboard for visulization of network graph and other important features. Use command tensorboard --logdir=graphs to get a feel of this notebook tensorboard. 
\

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import math
import time

In [0]:
# Step 1: Read in Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/mnist',one_hot=True)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [0]:
run_var=0

In [0]:
# To avoid haveing different graphs colliding
tf.reset_default_graph()
learning_rate = 0.001
batch_size = 128
n_epochs = 1
lamda = 0.0001
weight_initializer_xavier = tf.contrib.layers.xavier_initializer()
# Neurons in each layer
X_shape_0 = 784 # X.shape[0]
n_outputs = 10
Neurons_1 = 100
Neurons_2 = 50


In [0]:
def NN_Layer(input_1,size_in,size_out,name='Layer',activation='relu'):
    with tf.name_scope(name):
        w = tf.Variable(weight_initializer_xavier(shape=[size_in,size_out]),name='w') 
        b = tf.Variable(tf.zeros([1,size_out]),name='b')
        z = tf.matmul(input_1,w) + b
        if(activation=='relu'):
            a = tf.nn.relu(z)
        elif(activation=='sigmoid'):
            a = tf.nn.sigmoid(z)
        else:
            print('Error: Define Activation function again, carefully!')
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", a)
        return a
    

# function creates graph of NN
def NeuralNets(learning_rate):    
    #plcae holders for input: 
    X = tf.placeholder(tf.float32,[None,X_shape_0],name='X_placeholder')
    Y = tf.placeholder(tf.float32,[None,n_outputs],name='Y_placeholder')
    x_image=tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    # Layer 1
    layer_1 = NN_Layer(X,X_shape_0,Neurons_1,name='Layer_1')
    # Layer 2
    layer_2 = NN_Layer(layer_1,Neurons_1,Neurons_2,name='Layer_2') 
    #Output Layer
    layer_last = NN_Layer(layer_2,Neurons_2,n_outputs,name='Layer_last',activation='sigmoid')
                                   
    
    # specify cost function
    with tf.name_scope('entropy'):
    # this is our cost
        entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=layer_last,name='entropy')
        loss = tf.reduce_mean(entropy)
    # specify optimizer
    with tf.name_scope('train'):
        # optimizer is an "operation" which we can execute in a session
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('Accuracy'):
        # Accuracy
        correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(layer_last,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                                   
    # create a summary for our cost and accuracy
    tf.summary.scalar("cost", loss)
    tf.summary.scalar("accuracy", accuracy)                               
                                                                                          
    summary_op = tf.summary.merge_all()
    return summary_op,optimizer,loss    

In [0]:
# Run Session
log_dir = './graphs/TensorboardTUT/Adam/'
tf.reset_default_graph()
#Define graph
graph1 = tf.Graph()
with graph1.as_default():
    summary_op,optimizer,loss = NeuralNets(learning_rate)
    with tf.Session(graph=graph1) as sess:
        # to save the graph
        #writer = tf.summary.FileWriter('./graphs/TensorboardTUT/',sess.graph)
        writer = tf.summary.FileWriter('%s/%s' % (log_dir, run_var), sess.graph)
        #start_time = time.time()
        sess.run(tf.global_variables_initializer())
        n_batches = int(mnist.train.num_examples/batch_size)

        #Training 
        for epoch in range(n_epochs):
            total_loss = 0
            # Passing through each batch
            for j in range(n_batches):
                X_batch,Y_batch = mnist.train.next_batch(batch_size)
                _,loss_batch,summary = sess.run([optimizer,loss,summary_op],feed_dict={X:X_batch,Y:Y_batch})
                total_loss +=loss_batch
                writer.add_summary(summary, epoch * n_batches + j)


            print('Average loss epoch {0}:{1}'.format(epoch,total_loss/n_batches))    
        #print('Total_time:{0} seconds',format(time.time()-start_time))

        # Testing
        n_batches = int(mnist.test.num_examples/batch_size)
        total_correct_preds = 0
        for i in range(n_batches):
            X_batch, Y_batch = mnist.test.next_batch(batch_size)
            accuracy_batch = sess.run(accuracy,feed_dict={X:X_batch,Y:Y_batch})
            total_correct_preds += accuracy_batch
        print('Accuracy{0}'.format(total_correct_preds))    #/mnist.test.num_examples
        run_var+=1
    writer.close()    

NameError: name 'X' is not defined